# LATAM Airlines

Scrapear el sitio de Despegar para averiguar datos de vuelos en función del **origen** y **destino**, **fecha** y **cabina**. La información que esperamos obtener de cada vuelo es:

- Precios(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**NOTA**: en caso de no poder acceder (status_code = 403), buscar un sitio similar.

www.edestinos.com

In [1]:
import requests
from bs4 import BeautifulSoup

In [70]:
#url = 'https://www.edestinos.com/flights/select/oneway/ap/mvd/ap/for?departureDate=2020-12-10&pa=1&py=0&pc=0&pi=0&sc=economy'

In [27]:
url = 'https://www.latam.com/es_uy/apps/personas/booking?fecha1_dia=04&fecha1_anomes=2020-11&from_city1=MVD&to_city1=MAD&ida_vuelta=ida_vuelta&fecha2_dia=21&fecha2_anomes=2020-11&from_city2=MAD&to_city2=MVD&cabina=Y&nadults=1&nchildren=0&ninfants=0&app=deal-finder#/'

In [28]:
r = requests.get(url)

In [29]:
r.status_code

403

In [30]:
soup = BeautifulSoup(r.text, 'lxml')

In [31]:
# Importar Selenium (webdriver)

from selenium import webdriver

In [32]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')

driver = webdriver.Chrome(executable_path='chromedriver', options=options)


In [33]:
driver.get(url)

In [34]:
# Extraer informacion de la pagina

vuelos = driver.find_elements_by_xpath('//li[@class="flight"]') # // buscar en todo el arbol
len(vuelos)

5

In [35]:
vuelo = vuelos[0]

In [36]:
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="03899af6faa4aed37a8c309863cb0388", element="4f873ec1-6b92-42db-888f-fc4c5aa018dd")>

**Extrayendo horarios y duración**

In [37]:
#Hora de partida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime') # . buscar desde allí hacia abajo

'11:18'

In [38]:
#Hora de llegada
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')


'12:50'

In [39]:
# Duración de vuelo
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT21H32M'

**Contando escalas e interactuando con elementos del sitio**

In [40]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')

In [41]:
boton_escalas.click() #click en boton

In [42]:
# Segmentos del contenido en donde se ven las escalas
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
# Sin '.' porque busco todo el album.
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="03899af6faa4aed37a8c309863cb0388", element="411ec2be-be6b-42ce-b0d9-f6a69ab079e2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="03899af6faa4aed37a8c309863cb0388", element="e90cf7cd-a52f-46b8-9d2c-1c2a5f058859")>]

In [43]:
escalas = len(segmentos) - 1 # Porque cuenta desde salida/escala y escala/salida
print(f'El vuelo de MVD a MAD tiene {escalas} escala(s).')

El vuelo de MVD a MAD tiene 1 escala(s).


**Desafío: Escalas**

- **Escala #1**

In [44]:
segmento = segmentos[0]

In [45]:
# Aeropuerto de salida
segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text 


'Carrasco Intl.'

In [46]:
# Hora de salida
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'11:18'

In [47]:
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/span[@class="sr-only"]').text

'Salida a las 11 Horas 18 Minutos, de Carrasco Intl.'

In [48]:
# Duración de vuelo a próxima escala
segmento.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')

'2:32'

In [49]:
try:    
    combinacion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-label sc-hORach NXcGo"]').text
    cambio_avion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-changes sc-hORach NXcGo"]').text
    print(f'{combinacion} -- {cambio_avion}')
except:
    print('Revise XPath.')
    pass



Conexión en Santiago de Chile -- (cambio de avión)


In [58]:
modelo_avion = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text
flota = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/b').text

print(f'Flota {flota} - {modelo_avion}')

Flota LA405 - Airbus 320-200


- **Escala #2**

In [59]:
segmento_2 = segmentos[1]

In [60]:
# Aeropuerto de salida
segmento_2.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text 

'A. Merino Benítez Intl.'

In [61]:
# Hora de salida
segmento_2.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'19:55'

In [62]:
segmento_2.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/span[@class="sr-only"]').text


'Salida a las 19 Horas 55 Minutos, de A. Merino Benítez Intl.'

In [63]:
# Duración de vuelo a próxima escala
segmento_2.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')

'12:55'

In [85]:
modelo_avion = segmento_2.find_elements_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text
#flota = segmento_2.find_element_by_xpath('//div[@class="airline-flight-details"]/b').text



StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=85.0.4183.102)


- **Tarifas**

In [65]:
driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()

In [66]:
vuelo.click()

In [67]:
divisas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="currency-symbol"]')
valores = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="value"]')

divisas

[<selenium.webdriver.remote.webelement.WebElement (session="03899af6faa4aed37a8c309863cb0388", element="1c2cf6eb-d519-4965-9736-d10748910e88")>,
 <selenium.webdriver.remote.webelement.WebElement (session="03899af6faa4aed37a8c309863cb0388", element="17337ee3-98d1-4fe0-913e-3b5f67f61abf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="03899af6faa4aed37a8c309863cb0388", element="27763bbf-081f-4c3c-a59b-1a86db84735b")>]

In [68]:
divisa = divisas[0]
divisa.text

'US$'

In [69]:
moneda = valores[0]
print(divisa.text + moneda.text)

US$332


In [70]:
caca = []

for d in divisas:
    for v in valores:
        precio = d.text + ' ' + v.text
        caca.append(precio)
    break


tarifas = {
    'Light':caca[0],
    'Plus':caca[1],
    'Top':caca[2]
}
tarifas

{'Light': 'US$ 332', 'Plus': 'US$ 385', 'Top': 'US$ 445'}

In [73]:
vuelo.click()

In [74]:
def _get_tarifa(vuelo):
	divisas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="currency-symbol"]')
	valores = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="value"]')

	precios = []

	for d in divisas:
		for v in valores:
			precio = d.text + ' ' + v.text
			precios.append(precio)
		break
	

	tarifas = {
		'Light':precios[0],
    	'Plus':precios[1],
    	'Top':precios[2]
		}

	return tarifas

def _get_escalas(vuelo):
	# Segmentos del contenido en donde se ven las escalas
	segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
	escalas = len(segmentos) - 1 # Porque cuenta desde salida/escala y escala/salida
	print(f'El vuelo de MVD a MAD tiene {escalas} escala(s).')

	info_escalas = []
	for segmento in segmentos:
		# Aeropuerto de salida
		airport = segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text 
		# Hora de salida
		h_salida = segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')
		# Duración de vuelo a próxima escala
		duracion = segmento.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')
		# Info de combinación
		try:    
		    combinacion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-label sc-hORach NXcGo"]').text
		    cambio_avion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-changes sc-hORach NXcGo"]').text
		    info = f'{combinacion} -- {cambio_avion}'
		except:
		    print('Revise XPath.')
		    pass
		# Flota
		flota = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/b').text
		# Modelo de avón
		modelo_avion = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text

		data_escalas = {
			'Aeropuerto': airport,
			'Hora de salida': h_salida,
			'Duración p/ escala': duracion,
			'Info de conexión': info,
			'Flota': flota,
			'Avión': modelo_avion
		}
		info_escalas.append(data_escalas)

	return info_escalas

def _get_tiempos(vuelo):
	
	#Hora de partida
	h_salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
	#Hora de llegada
	h_llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
	# Duración de vuelo
	duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

	data_tiempos = {
		'Hora de salida':h_salida,
		'Hora de llegada':h_llegada,
		'Duración':duracion
	}
	return data_tiempos

def main_get_info(driver):
	
	# Extraer informacion de la pagina
	vueles = vuelos
	print(f'Se encontraron {len(vuelos)} vuelos.')
	print('Iniciando scraping...')
	info = []
	for vuelo in vueles:
		tiempos = _get_tiempos(vuelo)
		
		# Click en modal para conocer escalas
		boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
		boton_escalas.click()

		# Obtener escalas
		escalas = _get_escalas(vuelo)
		
		# Cerrar modal
		cerrar_modal = driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]')
		cerrar_modal.click()
		vuelo.click()

		# Obtener tarifas
		tarifas = _get_tarifa(vuelo)

		data_info = {
			'Tiempos':tiempos,
			'Escalas':escalas,
			'Tarifas':tarifas
		}
		info.append(data_info)

	return info


In [75]:
LATAM_info = main_get_info(driver)
LATAM_info

Se encontraron 5 vuelos.
Iniciando scraping...
El vuelo de MVD a MAD tiene 1 escala(s).
El vuelo de MVD a MAD tiene 2 escala(s).
El vuelo de MVD a MAD tiene 2 escala(s).
El vuelo de MVD a MAD tiene 2 escala(s).
El vuelo de MVD a MAD tiene 2 escala(s).


[{'Tiempos': {'Hora de salida': '11:18',
   'Hora de llegada': '12:50',
   'Duración': 'PT21H32M'},
  'Escalas': [{'Aeropuerto': 'Carrasco Intl.',
    'Hora de salida': '11:18',
    'Duración p/ escala': '2:32',
    'Info de conexión': 'Conexión en Santiago de Chile -- (cambio de avión)',
    'Flota': 'LA405',
    'Avión': 'Airbus 320-200'},
   {'Aeropuerto': 'A. Merino Benítez Intl.',
    'Hora de salida': '19:55',
    'Duración p/ escala': '12:55',
    'Info de conexión': 'Conexión en Santiago de Chile -- (cambio de avión)',
    'Flota': 'LA405',
    'Avión': 'Airbus 320-200'}],
  'Tarifas': {'Light': ' 332', 'Plus': 'US$ 385', 'Top': 'US$ 445'}},
 {'Tiempos': {'Hora de salida': '18:37',
   'Hora de llegada': '13:10',
   'Duración': 'PT38H33M'},
  'Escalas': [{'Aeropuerto': 'Carrasco Intl.',
    'Hora de salida': '18:37',
    'Duración p/ escala': '2:33',
    'Info de conexión': 'Conexión en Santiago de Chile -- (cambio de avión)',
    'Flota': 'LA409',
    'Avión': 'Airbus 320-200'}

In [80]:
import pandas as pd

df = pd.DataFrame(LATAM_info)
df.to_excel('LATAM MVD a MAD 2020.xls')

In [86]:
driver.close()